In [7]:
import numpy as np
import tensorflow as tf
import os
import pandas as pd

In [8]:
def extract_columns (df, columns):
    new_df = pd.DataFrame()
    for c in columns:
        if c in df.columns:
            new_df[c] = df[c]
        else:
            new_df[c] = np.nan
    return new_df

In [42]:
in_path = os.path.join("data","2_cleaned",'horses')
out_path = os.path.join("data","3_extracted")

for file_name in [f for f in os.listdir(in_path) if not f.startswith(".")]:

    # try:
    #     df = pd.read_csv(os.path.join(out_path, file_name), index_col=0)
    #     if 'jockey_age' in df.columns:
    #         continue
    # except:
    #     pass

    horse_df = pd.read_csv(os.path.join(in_path, file_name), index_col=0)
    if 'horse_finish_time' not in horse_df.columns:
        continue
    for c in horse_df.columns:
        horse_df[c] = horse_df[c].apply(lambda x: str(x).lower())
    horse_df['horse_name'] = file_name.split('(')[0].strip()
    horse_df['finish_time'] = horse_df['horse_finish_time']
    horse_df['race_date'] = pd.to_datetime(horse_df['race_date'], format='%Y/%m/%d', errors='coerce')
    horse_df['race_index'] = horse_df['race_date'].apply(lambda x: x.strftime('%Y_')) + pd.to_numeric(horse_df['Race Index']).astype(int).astype(str)
    horse_df.drop_duplicates('race_date', inplace=True)
    horse_df = horse_df[horse_df['horse_finish_time'] != '--']
    horse_df.reset_index(drop=True, inplace=True)
    if 'Import Date' in horse_df.columns:
        horse_df['Import Date'] = pd.to_datetime(horse_df['Import Date'], format='%d/%m/%Y', errors='coerce')
        horse_df['days_since_import'] = (horse_df['race_date'] - horse_df['Import Date']).apply(lambda x: x.days)
    else:
        earliest_race_date = horse_df['race_date'].min()
        horse_df['days_since_import'] = (horse_df['race_date'] - earliest_race_date).apply(lambda x: x.days + 177)
    horse_df['horse_gear'] = (horse_df['Gear'] != '--').astype(int)
    horse_df['total_stakes'] = horse_df['Total Stakes*'].apply(lambda x: x.replace('$','').replace(',',''))

    jockey_age = []
    jockey_nationality = []
    jockey_wins = []
    jockey_rides = []
    jockey_stakes = []
    jockey_same_race_wins = []

    for idx, row in horse_df.iterrows():
        jockey_file_path = os.path.join('data','1_scrape','jockeys',f'{row["Jockey"]}.csv')
        if not os.path.exists(jockey_file_path):
            jockey_age.append(np.nan)
            jockey_nationality.append(np.nan)
            jockey_wins.append(np.nan)
            jockey_rides.append(np.nan)
            jockey_stakes.append(np.nan)
            jockey_same_race_wins.append(np.nan)
            continue
        jockey_df = pd.read_csv(jockey_file_path, index_col=0)
        try:
            jockey_age.append(int(jockey_df['jockey_age'].unique()[0]) - (2023 - int(row['race_date'].year)))
        except:
            jockey_age.append(np.nan)
        jockey_nationality.append(jockey_df['nationality'].unique()[0])
        jockey_wins.append(jockey_df['no. of wins'].unique()[0])
        jockey_rides.append(jockey_df['total rides'].unique()[0])
        jockey_stakes.append(str(jockey_df['stakes won'].unique()[0]).replace('$','').replace(',','').strip())

        race_distance = str(row["race_distance"]).replace('.0','')
        if 'sha tin' in row["race_location"]:
            try:
                jockey_same_race_wins.append(jockey_df[f'{row["race_surface"]}_{race_distance}_win'].unique()[0])
            except:
                jockey_same_race_wins.append(jockey_df[f'sha tin_all weather_{race_distance}_win'].unique()[0])
        else:
            jockey_same_race_wins.append(jockey_df[f'{row["race_location"].replace("ch","conghua")}_{race_distance}_win'].unique()[0])

    horse_df['jockey_age'] = jockey_age
    horse_df['jockey_nationality'] = jockey_nationality
    horse_df['jockey_wins'] = jockey_wins
    horse_df['jockey_rides'] = jockey_rides
    horse_df['jockey_stakes'] = jockey_stakes
    horse_df['jockey_same_race_wins'] = jockey_same_race_wins

    df = extract_columns(horse_df, ['race_index','race_date','total_stakes','horse_name','horse_weight','horse_handicap','horse_odds','horse_rating','horse_import_type','horse_sex','horse_colour',
                'horse_age','horse_origin','horse_gear','days_since_import',
                'draw',
                'jockey_age','jockey_nationality','jockey_wins','jockey_rides','jockey_stakes','jockey_same_race_wins',
                'race_location','race_class','race_going','race_distance','race_surface','race_course',
                'finish_time','place'])

    df.to_csv(os.path.join(out_path, file_name))
    print(file_name, 'saved')
    

strongest boy(h027).csv saved
le plus vite(g242).csv saved
spicy grill(d159).csv saved
ever blessing(h187).csv saved
dorado fig(g332).csv saved
coming patch(g222).csv saved
super wealthy(d108).csv saved
to infinity(g277).csv saved
general winner(e376).csv saved
nearly fine(e430).csv saved
podium(h173).csv saved
gift of lifeline(c172).csv saved
winwin thirtythree(c110).csv saved
horse of good luck(h160).csv saved
ka ying glorious(h026).csv saved
capital delight(h262).csv saved
touch faith(e153).csv saved
nordic star(g433).csv saved
mark the moment(h088).csv saved
flagship warrior(g452).csv saved
silver fig(b025).csv saved
super wise dragon(g475).csv saved
goal oriented(g254).csv saved
super eagle(g408).csv saved
amazing victory(g123).csv saved
young explorer(g084).csv saved
hei wongchoy(g187).csv saved
sky forever(e414).csv saved
one step ahead(c242).csv saved
call me teddy(g233).csv saved
our class act(e303).csv saved
beauty crescent(h334).csv saved
vanquisher(h131).csv saved
family fo

In [43]:
in_path = os.path.join("data","3_extracted")

entire_df = pd.DataFrame()

for file_name in [f for f in os.listdir(in_path) if not f.startswith(".")]:

    file = os.path.join(in_path, file_name)
    current_race_df = pd.read_csv(file, index_col=0)

    entire_df = pd.concat([entire_df, current_race_df])

/var/folders/vq/8tw_x9796214x5ztxkg8rkt80000gn/T/ipykernel_12177/2953364818.py:10: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  entire_df = pd.concat([entire_df, current_race_df])
/var/folders/vq/8tw_x9796214x5ztxkg8rkt80000gn/T/ipykernel_12177/2953364818.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  entire_df = pd.concat([entire_df, current_race_df])
/var/folders/vq/8tw_x9796214x5ztxkg8rkt80000gn/T/ipykernel_12177/2953364818.py:10: FutureWarning: The behavior of array concatenation with empty entries is deprecated. 

In [52]:
entire_df.to_csv(os.path.join('data','all_races.csv'))

In [48]:
out_path = os.path.join("data","4_races")

for race_index in entire_df['race_index'].unique():
    df = entire_df[entire_df['race_index'] == race_index]
    if len(df.index) < 5:
        continue
    df.sort_values('place', inplace=True)

    df.to_csv(os.path.join(out_path,f'{race_index}.csv'))

/var/folders/vq/8tw_x9796214x5ztxkg8rkt80000gn/T/ipykernel_12177/2011453047.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values('place', inplace=True)


In [53]:
df = pd.read_csv(os.path.join('data','all_races.csv'), index_col=0)

df = df[df['place'].notna()]

for c in df.columns:
    try:
        df[c] = df[c].astype('float32')
    except:
        pass


In [54]:

mean_std_dict = {}

for c in df.columns:
    if c == 'race_index':
        continue

    if df[c].dtype != 'float32':
        df[c].replace({"":np.nan}, inplace=True)
        df[c].fillna(df[c].mode().iloc[0], inplace=True)
        continue

    df[c] = pd.to_numeric(df[c], errors='coerce')

    mean = df[c].mean()
    std = df[c].std()

    df[c] = df[c].replace(np.nan, mean)
    
    mean_std_dict[c] = {'mean': mean, 'std': std}

mean_std_df = pd.DataFrame(mean_std_dict)
mean_std_df.to_csv(os.path.join('data', 'mean_std.csv'))

df.to_csv('data/all_races_cleaned.csv')

In [55]:
for c in df.columns:
    print(c, df[c].isna().any())

race_index False
race_date False
total_stakes False
horse_name False
horse_weight False
horse_handicap False
horse_odds False
horse_rating False
horse_import_type False
horse_sex False
horse_colour False
horse_age False
horse_origin False
horse_gear False
days_since_import False
draw False
jockey_age False
jockey_nationality False
jockey_wins False
jockey_rides False
jockey_stakes False
jockey_same_race_wins False
race_location False
race_class False
race_going False
race_distance False
race_surface False
race_course False
finish_time False
place False
